In [1]:
from sage.combinat.permutation import Permutations_mset
from sage.combinat.shuffle import SetShuffleProduct
from sage.matrix.args import MatrixArgs_init
from sage.matrix.args import SparseEntry

s = SymmetricFunctions(QQ).schur()
    
# This function computes the dimension of the cokernel of the last differential of the appropriate line.
# Since the Euler characteristic is known we also obtain the other homology group.
# 'degrees' is actually two lists [lambda, mu] and it tells the computer to give the (integral) multiplicity of
# Sym^{lambda_i}(QQ*alpha_{2i+1}) x Sym^{mu_j}(QQ*alpha_{2j+2}) (i, j ranging over the indices of lambda resp. mu)
def Compute_Rank(particles, degrees, Show_Progress = False):
    line = sum(degrees[0]) + sum(degrees[1])
    pos_degree = pos_degree_classes(degrees)
    
    eul = sum((-1)^(particles-line+col)*dim_E1(particles, line, col, pos_degree) for col in range(particles-line+1))
    if Show_Progress:
        print(f"Difference of ranks: {eul}")
    
    col = particles-line-1

    domain = E1(particles, line, col, pos_degree)
    codomain = E1(particles, line, col+1, pos_degree)
    len_domain = dim_E1(particles, line, col, pos_degree)
    len_codomain = dim_E1(particles, line, col+, pos_degree)
    
    mat = diff1(len_domain, len_codomain, domain, codomain, Show_Progress = Show_Progress)
    if Show_Progress:
        print("Computing the rank of the differential...", end = "")
    H_top = len_codomain - mat.rank()
    if Show_Progress:
        print("......done!")

    H_bottom = H_top - eul
    if Show_Progress:
        actual_line = sum(val*(2*i+1) for i, val in enumerate(degrees[0]))
        actual_line += sum(val*(2*i+2) for i, val in enumerate(degrees[1]))
        print(f"In dimension {actual_line+col} :", H_top - eul)
        print(f"In dimension {actual_line+col+1}:", H_top)
        
    return H_bottom, H_top

#Computes the S_n multiplicity of the irreducible attached to 'degrees', as described in the previous function
def Multiplicity(degrees):
    KoszulSign = [-s[1] for el in degrees[0]] + [s[1] for el in degrees[1]]
    
    SymFunc = univ_graded(KoszulSign)
    tot_degree = sum(degrees[0]) + sum(degrees[1])
    
    return Focus_GL(s[tot_degree].plethysm(SymFunc), [[el] for el in degrees[0]] + [[el] for el in degrees[1]])
    
################################# Helper functions ###########################################################    

## Computation for (a basis of) the vector spaces Lie(T) = Lie(k_1) x Lie(k_2) x ... x Lie(k_p)
## The basis is computed by the second function; the first function collates the basis for all T of the same length
## A basis element is represented by a forest whose leaves are labelled by the numbers 1,2,...,n, each of them used
## exactly once; each (rooted) tree corresponding to a Lie word in Lie(k_i).

# returns a tuple
def list_trees(particles, col):
    L = LieAlgebra(QQ, 'x', particles)
    list_gens = [key.to_word()[0] for el in L.gens() for key, val in el]

    pattern = (el for part in Partitions(particles, length = particles-col)
                  for el in SetPartitions(list_gens, part))
            
    result = []
    for el in pattern:
        result += cartesian_product([LieOperad(L, part) for part in el])
    return tuple(result)

def LieOperad(LieAlg, variables):
    Basis = LieAlg.Lyndon()
    return tuple(filter(lambda el : all((set(key.to_word()) == variables for key, _ in el)),
                        Basis.graded_basis(len(variables))))

## Computation for (a basis of) H(X)^k, or rather of the summand for the chosen irreducible representation

def pos_degree_classes(degrees):
    pattern = []
    for index, val in enumerate(degrees[0]):
        pattern += [2*index+1]*val
    for index, val in enumerate(degrees[1]):
        pattern += [2*index+2]*val
    return tuple(Permutations_mset(pattern))

# returns a generator
def list_homology(particles, line, col, Basis):
    #return (el for pos_deg in Basis
    #           for el in ShuffleProduct(tuple(pos_deg), [0]*(particles-line-col)))
    #return chain.from_iterable(ShuffleProduct(tuple(pos_deg), [0]*(particles-line-col))
    #                                     for pos_deg in Basis)
    return SetShuffleProduct(([0]*(particles-line-col),), Basis)

## Computation of the terms on the E1 page

# returns a generator
def E1(particles, line, col, VectSpace): 
    trees = list_trees(particles, col)
    return (tuple(zip(tree, gen)) for gen in list_homology(particles, line, col, VectSpace) 
                                  for tree in trees)

# returns the dimension of the term E_1^{p,q}
def dim_E1(particles, line, col, VectSpace):
    return stirling_number1(particles, particles-col)*len(VectSpace)*binomial(particles-col, line)

## Computation of the differential

# returns a matrix
def diff1(len_domain, len_codomain, domain, codomain, Show_Progress = False):
    images = {el : [] for el in codomain}
    
    if Show_Progress:
        print("Computing differential:...", end = "")
    
    for i, basis_el in enumerate(domain):
        pos, neg = find_image(basis_el)
        for p in pos:
            images[p].append((i, 1))
        for n in neg:
            images[n].append((i, -1))
            
    if Show_Progress:
        print("...done!")
   
    if Show_Progress:
        print("Computing (sparse) matrix entries:...", end = "")
    
    entries = [SparseEntry(i, index_dom, val) for i, pairs in enumerate(images.values())
                                              for index_dom, val in pairs]
    
    del images
    if Show_Progress:
        print("..................done!")
    
    S = MatrixSpace(ZZ, len_codomain, len_domain, sparse = True)
    return MatrixArgs_init(S, entries).matrix()

def find_image(domain_el):
    pos = []
    neg = []
    
    for left, right in Combinations(range(len(domain_el)), 2):
        pos_temp, neg_temp = cup_product(domain_el, left, right)
        pos.extend(pos_temp)
        neg.extend(neg_temp)
   
    return pos, neg

def cup_product(domain_el, left, right):
    tree_left, gen_left = domain_el[left]
    tree_right, gen_right = domain_el[right]
    
    if gen_left > 0 and gen_right > 0:
        return [], []
    
    # finds the sign coming from the Koszul rule
    if gen_left % 2 == 0 and gen_right % 2 == 1:
        sign = 1
    else: 
        sign = -1 # if gen_left%2 == 1, the sign comes from permuting the cohomology class with the Lie word
                  # if gen_right%2 == 0 == gen_left%2, the second loop makes one more sign flip than it should
    
    if gen_left %2 == 0:
        for _, gen in domain_el[:left]:
            if gen %2 == 0:
                sign *= -1
    
    if gen_right %2 == 0:
        for (_, gen) in domain_el[:right]:
            if gen %2 == 0:
                sign *= -1
    
    # computes the bracket of the Lie words and filters the terms according to their sign
    prod = tree_left.bracket(tree_right)
#    result = [el for i, el in enumerate(domain_el) if i != left and i != right]
            
#    return [add_image([(L({key:1}), gen_left+gen_right)] + result, sign*val)
#            for key, val in prod.monomial_coefficients().items()]
    pos = []
    neg = []
    for key, val in prod.monomial_coefficients().items():
        el, sign_el = add_image(prod.parent()({key:1}),
                                gen_left+gen_right, #if we get here, at least one of the summands is 0
                                [el for i, el in enumerate(domain_el) if i != left and i != right],
                                sign*val)
        if sign_el > 0:
            pos.append(el)
        else: #if sign_el < 0
            neg.append(el)
        
    return pos, neg

# Finds the position where to insert the new Lie word and what is the Koszul sign associated to moving the term there
# Maybe evaluating gen % 2 only once would improve performance, but it would significantly degrade readability
def add_image(key, gen, result, sign):
    pos = 0
    for key_other, gen_other in result:
        if key < key_other:
            break
        pos += 1
        if gen % 2 == 0 and gen_other % 2 == 0:
            sign *= -1

    result.insert(pos, (key, gen))
    return tuple(result), sign

############ Helper functions for 'Multiplicity' ##############################
def univ_graded(list_signs):
    result = list_signs[0]
    for el in list_signs[1:]:
        result = result.tensor(s.one()) + (result.parent()(1)).tensor(el)
        
    return s[1].tensor(result)

def Focus_GL(SymTensor, list_degrees):
    return sum(coeff*s[index[0]]*all(index[i+1] == list_degrees[i] for i in range(len(list_degrees)))
               for index, coeff in SymTensor)

In [ ]:
particles = 10
degrees = [[], [5,2,1]]
Compute_Rank(particles, degrees, Show_Progress = True)

print()
print(f"Corresponding S_n multiplicity : {Multiplicity(degrees)}")